In [1]:
import pickle
import pandas as pd
import os
import csv
import shutil
import collections
import csv
from IPython.core.display import HTML
import gensim
import pickle
import nltk
import os
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, join
#nltk.download()
import string 
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
import collections 

pd.set_option('display.max_rows', None)

with open('/data/WorkData/firmEmbeddings/CaseData/id2gram.pkl', 'rb') as f:
    listofgrams = pickle.load(f)
print(len(listofgrams))

#case metadata
casedata = pd.read_pickle('/data/WorkData/firmEmbeddings/CaseData/casedata.pkl')

#taking only year 2013
yearr="2013"
df=casedata[(casedata["Year"] == yearr)]
print(len(df))

#list of n grams present in firm related cases of 2013
listofgrams_law={}
 
for i in range(1,13):
    pathname = os.path.join("/data/WorkData/firmEmbeddings/CaseData/PickleFiles/"+yearr, str(i))
    files=os.listdir(pathname)
    for file in files:
        file1=file.split("_")[0]
        caseid=file1
        
        path=pathname+"/"+file
        with open(path, 'rb') as m:
            mapping = pickle.load(m)
       
        
        for sent in mapping:
            for token in sent:
                if token in listofgrams:
                    listofgrams_law[listofgrams[token]]=token
                else:
                    print("error")
print(len(listofgrams_law))
#listofgrams_law

96977
485
42586


In [3]:

#check df tf of firm related bigrams
fewgrams={}
with open('/data/WorkData/firmEmbeddings/CaseData/df-tf.pkl', 'rb') as f:
    dftf = pickle.load(f)
print(len(dftf))

df1=dftf['phrase'].tolist()
i=0
for bigram in df1:
    if bigram in  listofgrams_law:
        tt=dftf.loc[dftf['phrase'] == bigram, 'termfreqs'].item()
        
        fewgrams[bigram]=tt
    else:
        i=i+1
        
print(i)     
print(len(fewgrams))
#fewgrams

100495
57909
42586


In [4]:
#taking bigrams with specific dftf
finalgrams={}
for f in fewgrams:
    if fewgrams[f]<9000 and fewgrams[f]>800:
        finalgrams[f]=listofgrams_law[f]


print(len(finalgrams))
#finalgrams

12160


In [5]:
#final dataframe
columns1=['CaseId','Date','Month']


for f in finalgrams:
    columns1.append(f)


ngramdata= pd.DataFrame(columns=columns1)



In [8]:
#appending rows to dataframe for each case of year 2013 

i=0
for index, row in df.iterrows():
    i=i+1
    
    caseid=index
    date=row["Date"]
    month=int(row["Month"])
    
    ngramdata = ngramdata.append({'CaseId':caseid,'Date':date,'Month':month},ignore_index=True)
    
print(len(ngramdata))   

ngramdata = ngramdata.set_index('CaseId')


485


In [9]:

ngramdata.head()


,Date,Month,affidavit_state,different_view,case_differs,car_wash,aggravated_murder,review_committee,correct_result,additional_defendant,...,deposit_account,policy_state,regulation_promulgate,defendant_united,uniform_rule,seeking_relief,class_counsel,news_report,could_sell,enrichment_claim
CaseId,,,,,,,,,,,,,,,,,,,,,
X1HT4F0N,5/21/13,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X1JB7OQ003,2/28/13,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X1J8UPE003,2/7/13,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X1JEEKK003,3/25/13,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X1JSOJI003,7/12/13,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:



#reading pickle files and updating the ngrams that our present 
for i in range(1,13):
    pathname = os.path.join("/data/WorkData/firmEmbeddings/CaseData/PickleFiles/"+yearr, str(i))
    files=os.listdir(pathname)
    for file in files:
        file1=file.split("_")[0]
        caseid=file1
        
        path=pathname+"/"+file
        with open(path, 'rb') as m:
            mapping = pickle.load(m)
       
        
        for sent in mapping:
            for token in sent:
                if token in finalgrams.values():
                    
                    ngramdata.loc[caseid,listofgrams[token]]=1
                
                    
        





In [11]:

ngramdata=ngramdata.fillna(0)
ngramdata.head(10)

,Date,Month,affidavit_state,different_view,case_differs,car_wash,aggravated_murder,review_committee,correct_result,additional_defendant,...,deposit_account,policy_state,regulation_promulgate,defendant_united,uniform_rule,seeking_relief,class_counsel,news_report,could_sell,enrichment_claim
CaseId,,,,,,,,,,,,,,,,,,,,,
X1HT4F0N,5/21/13,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1JB7OQ003,2/28/13,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1J8UPE003,2/7/13,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1JEEKK003,3/25/13,3,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
X1JSOJI003,7/12/13,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1JKU4G003,5/13/13,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1GERHON,3/28/13,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1JB2P6003,2/27/13,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1JTL1Q003,7/18/13,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
ngramdata.to_pickle("/data/WorkData/firmEmbeddings/Models/StockPredictionUsingNeuralNetwork/ngramdata.pkl")




In [13]:
with open('/data/WorkData/firmEmbeddings/Models/StockPredictionUsingNeuralNetwork/ngramdata.pkl', 'rb') as f:
    ngramdata = pickle.load(f)


ngramdata.head(10)

,Date,Month,affidavit_state,different_view,case_differs,car_wash,aggravated_murder,review_committee,correct_result,additional_defendant,...,deposit_account,policy_state,regulation_promulgate,defendant_united,uniform_rule,seeking_relief,class_counsel,news_report,could_sell,enrichment_claim
CaseId,,,,,,,,,,,,,,,,,,,,,
X1HT4F0N,5/21/13,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1JB7OQ003,2/28/13,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1J8UPE003,2/7/13,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1JEEKK003,3/25/13,3,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
X1JSOJI003,7/12/13,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1JKU4G003,5/13/13,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1GERHON,3/28/13,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1JB2P6003,2/27/13,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X1JTL1Q003,7/18/13,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
print(len(ngramdata))

485
